In [9]:
import random
import scipy.io as sio
import operator
import itertools
import collections

# loading data
def loaddata(k):
    mat_contents = sio.loadmat('../tmp/dat-multi-'+str(k)+'.mat')
    oct_struct   = mat_contents['dat']
    X ,Y         = oct_struct['xx'], oct_struct['yy']
    X, Y         = X[0,0], Y[0,0]
    return X, Y
    
def readfile_name():
    with open('../data/All_app.tsv') as f:
        app_name = f.read().split('\n')
    return app_name

In [29]:
# job description
class job_slice:
    def __init__(self, j_name, j_iden, j_type, j_work, qos, j_deadline):
        self.name     = j_name     # job_name
        self.iden     = j_iden
        self.job      = j_type     # job_type: {qos, batch, low_priority}
        self.work     = j_work     # work: int
        self.qos      = qos        # qos: decimal between 0-1
        self.deadline = j_deadline # deadline: int
           
    def __repr__(self):
        statem1 = '{' + self.job +' '+ str(self.iden) +' '+ self.name\
                   +' W:'+ str(self.work) 
        statem2 = ' qos:'+ str(self.qos) + ' ddl:'+ str(self.deadline) +'}'
        return statem1 + statem2
    
class jobSched_slice:
    def __init__(self, j_name, j_iden, j_type, j_work, qos, j_deadline, \
                 j_speed, j_runtime):
        self.name     = j_name     # job_name
        self.iden     = j_iden
        self.job      = j_type     # job_type: {qos, batch, low_priority}
        self.work     = j_work     # work: int
        self.qos      = qos        # qos: decimal between 0-1
        self.deadline = j_deadline # deadline: int
        self.speed    = j_speed
        self.runtime  = j_runtime
    def __repr__(self):
        statem1 = '{' + self.job +' '+ str(self.iden) +' '+ self.name\
                   +' W:'+ str(self.work) 
        statem2 = ' qos:'+ str(self.qos) + ' ddl:'+ str(self.deadline) +'}'
        return statem1 + statem2
        

class schedule:
    def __init__(self):
        self.jobs = []
        self.time = 0
    def __repr__(self):
        return []
        
            
# generate a random job queue
def generate_jobqueue(N, P, qos, app_name):
    """
    :type N:          int: Number of jobs
    :type qos:        float: qos constraint
    :type app_name:   list[str]: application names
    :rtype: job_queue:list[job_slice]: Set of jobs
    """    
    # Harcoding qos job names
    app_name_qos   = ['x264','vips','swish','swaptions','streamcluster',\
                      'STREAM','fluidanimate','blackscholes']
    app_name_batch = [item for item in app_name if item not in app_name_qos]
    num_qos   = 0
    job_queue = []
    for i in range(N):
        j_work     = random.choice([100,500,1000])
        if num_qos < P/2:
            j_type     = random.choice(['qos', 'batch'])
        else:
            j_type     = 'batch'
            
        if j_type      == 'qos':
            j_deadline = None  
            j_qos      = qos
            num_qos    += 1
            indices    = [i for i,item in enumerate(app_name) if item in app_name_qos]
            j_iden     = random.choice(indices)
            j_name     = app_name[j_iden-1]
        elif j_type    == 'batch':
            j_deadline = random.choice([100, 500, 1000])
            j_qos      = None
            indices    = [i for i,item in enumerate(app_name) if item in app_name_batch] 
            j_iden     = random.choice(indices)
            j_name     = app_name[j_iden-1]
        job        = job_slice(j_name, j_iden, j_type,  j_work, j_qos,\
                               j_deadline)
        job_queue.append(job);
    #job_queue.sort(key=operator.attrgetter('job'));
    return job_queue

In [10]:
#description
job_slice = collections.namedtuple('job_slice',['name','iden','type','work',\
                                    'qos','deadline'])
# generate a random job queue
class jobqueue_random:
    def __init__(self, N, qos, app_name):
        """
        :type N:          int: Number of jobs
        :type qos:        float: qos constraint
        :type app_name:   list[str]: application names
        :rtype: job_queue:list[job_slice]: Set of jobs
        """    
        # Harcoding qos job names
        app_name_qos   = ['x264','vips','swish','swaptions','streamcluster',\
                      'STREAM','fluidanimate','blackscholes']
        app_name_batch = [item for item in app_name if item not in app_name_qos]
        num_qos   = 0
        job_queue = []
        for i in range(N):
            j_work     = random.choice([100,500,1000])
            if num_qos < P/2:
                j_type     = random.choice(['qos', 'batch'])
            else:
                j_type     = 'batch'

            if j_type      == 'qos':
                j_deadline = None  
                j_qos      = qos
                num_qos    += 1
                indices    = [i for i,item in enumerate(app_name) \
                              if item in app_name_qos]
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            elif j_type    == 'batch':
                j_deadline = random.choice([100, 500, 1000])
                j_qos      = None
                indices    = [i for i,item in enumerate(app_name) \
                              if item in app_name_batch] 
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            job = job_slice(j_name, j_iden, j_type,  j_work, j_qos, j_deadline)
            job_queue.append(job);
        #job_queue.sort(key=operator.attrgetter('job'));
        self._job_queue = job_queue
        
    def __len__(self): 
        return len(self._job_queue)
    
    def __getitem__(self, position): 
        return self._job_queue[position]

In [11]:
# prediction
def random_forest():
    return[]

def nchoosek_rep(array, k):
    return list(itertools.combinations_with_replacement(array, k))

In [13]:
N         = 10  # Number of jobs in the queue
P         = 4   # Number of processors
k         = 2   # Pairs of applications
qos       = 0.9 # QoS threshold

app_name  = readfile_name() 
X,Y       = loaddata(2)
n         = len(app_name)
#job_queue = generate_jobqueue(N, P, qos, app_name)
job_queue = jobqueue_random(N, qos, app_name)

In [16]:
# process job queue and return schedule
def process_jobqueue(P, Y, k, app_name, job_queue):
    """
    :type P:          int: Number of processors 
    :type Y:          list[list[float]]: Performance of application pairs
    :type job_queue:  list[job_slice]: Set of jobs to be scheduled
    :rtype: schedule: schedule
    """    
    
    n         = len(app_name)
    print('Jobs')
    for job in job_queue:
        print(job)
    
    job_list = sorted([job_item.iden for job_item in job_queue])
    a,b      = nchoosek_rep(job_list, 2), nchoosek_rep(range(n), 2)
    common_index = [i for i, e in enumerate(b) if e in a]
    
    
    
    return []
job_sched = process_jobqueue(P, Y, k, app_name, job_queue)


Jobs
{batch 2 backprop W:500 qos:None ddl:1}
{batch 4 blackscholes W:1000 qos:None ddl:0.5}
{batch 2 backprop W:1000 qos:None ddl:0.5}
{batch 6 cfd W:500 qos:None ddl:1}
{batch 5 btree W:1000 qos:None ddl:1}
{batch 25 swish W:100 qos:None ddl:0.5}
{qos 27 vips W:1000 qos:0.9 ddl:None}
{qos 24 swaptions W:100 qos:0.9 ddl:None}
{qos 24 swaptions W:1000 qos:0.9 ddl:None}
{qos 27 vips W:1000 qos:0.9 ddl:None}


In [33]:
for job in job_queue:
    print(job)

job_list = sorted([job_item.iden for job_item in job_queue])
a,b      = nchoosek_rep(job_list, 2), nchoosek_rep(range(n), 2)
common_index = [i for i, e in enumerate(b) if e in a]
print(common_index)
Y[common_index]

# random
# at each step check if we can squeeze in job into in any machine
numMachines = 0
machineStatus = []
for job in job_queue:
    for machine in machineStatus:
        #
        if :
        

{batch 2 backprop W:500 qos:None ddl:500}
{qos 20 srad W:500 qos:0.9 ddl:None}
{batch 12 hotspot W:100 qos:None ddl:100}
{batch 9 fluidanimate W:1000 qos:None ddl:500}
{batch 5 btree W:1000 qos:None ddl:1000}
{qos 3 bfs W:1000 qos:0.9 ddl:None}
{batch 6 cfd W:1000 qos:None ddl:500}
{batch 11 HOP W:1000 qos:None ddl:500}
{batch 15 lavaMD W:500 qos:None ddl:1000}
{batch 9 fluidanimate W:100 qos:None ddl:1000}
[55, 56, 58, 59, 62, 64, 65, 68, 73, 81, 83, 84, 87, 89, 90, 93, 98, 130, 131, 134, 136, 137, 140, 145, 153, 156, 158, 159, 162, 167, 216, 218, 219, 222, 227, 253, 254, 257, 262, 270, 273, 278, 315, 320, 370]


array([[ 0.92828749,  0.92828749],
       [ 0.41318297,  0.9633187 ],
       [ 0.53777159,  0.91322071],
       [ 0.91562811,  0.95697845],
       [ 0.97054057,  0.96550565],
       [ 0.85217065,  0.73924021],
       [ 0.525111  ,  0.3777129 ],
       [ 0.69512986,  0.96687591],
       [ 0.47782032,  0.74245462],
       [ 0.5071318 ,  0.5071318 ],
       [ 0.84357964,  0.20619974],
       [ 0.88856105,  0.5023819 ],
       [ 0.94372971,  0.33654608],
       [ 0.92023591,  0.2123618 ],
       [ 0.85301442,  0.07238739],
       [ 0.77968538,  0.40014071],
       [ 0.68356183,  0.30332568],
       [ 0.19207579,  0.19207579],
       [ 0.21840899,  0.81809276],
       [ 0.24121014,  0.8427604 ],
       [ 0.52338425,  0.36672745],
       [ 0.56464258,  0.23919045],
       [ 0.36446326,  0.64259733],
       [ 0.35049567,  0.53109865],
       [ 0.68172556,  0.68172556],
       [ 0.83767189,  0.50831861],
       [ 0.94520513,  0.90604839],
       [ 0.87727403,  0.84321355],
       [ 0.79173789,